The following code provide the general workflow of gene-set enrichment analysis;
Here we used the overrepresentation analysis

    1) Once PRS and phenotype association has been confirmed. We select the $P_{T}$ threshold at which there is the PRS is most strongly associated with the phenotype of interest.
    2) Next we select the SNPs contributing to that PRS at that $P_{T}$
    3) We perform single univariate analysis for each one of the SNP with the phenotype. And select an arbitrary threshold of 0.05 as the cutoff for the SNPs. The SNPs are then reasoned to be strongly associated with both the disorder and the phenotype of interest.
    4) SNPs are then mapped to genes, within the starting and stoping position of the gene.
    5) The gene list is then tested for enrichment using FUMA code.
    6) We randomly select n number of SNPs from step 2, and generate gene list and perform FUMA to see whether the process of doing single univariate analysis gives us a different set of pathways than from random alone.


In [1]:
import sys
sys.path.append('../../')
sys.path.append('../')


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap

In [2]:
from codes.docs.analysis.nimagen import stats, visualisation, graph,genes
from codes.docs.analysis import data_preprocessing, genetic_file_preprocess

The .snp file provided by PRSice-2 will generally hold the SNPs after LD-based clumping. Those are the ones you want to do GSEA with.

In [8]:
def create_random_genesets(job,n_sample,bed_file,output_folder='.'):
    random_SNPs = best_SNPs.sample(n_sample)
    random_SNPs_annot = genes.SNPsFunctionalAnalysis(
        snps_list=random_SNPs.SNP.to_list(),
        bed_file=
        bed_file
    )
    random_SNPs_annot.genes_ID, random_SNPs_annot.snp_ID = random_SNPs_annot.SNPs_annotation_to_gene(
        snps_list=random_SNPs.SNP.tolist(),
        gene_build_path=
        f'../dataset/genetic_dataset/gene_build/NCBI37.3.gene.loc',
        window_size=0)
    random_SNPs_annot.genes_ID[['Gene_ID']].to_csv(
        f'{output}/random_genes_all{job}.txt',
        header=False,
        index=False,
        sep=' ')

In [9]:
def generate_best_enriched_path(job,input_folder='.',ouput_folder='.'):
    msigdb_dataset ='../dataset/genetic_dataset/pathway_database/MSigDB/MSigDB_custom_entrez.gmt'
    background_gene = '../dataset/genetic_dataset/gene_build/NCBI37_gene_loc.txt'
    gene_job = f'{input_folder}/random_genes_all{job}.txt'
    res = genes.GeneSetEnrichment.ora(msigdb_dataset,background_gene,gene_job,disable_tqdm=True)
    res['run'] = job
    res = res.sort_values(by='adjP')
    output_file = f'{output_folder}/random_enriched_paths.csv'
    if not os.path.isfile(output_file):
        res.head(3).to_csv(output_file, header='column_names')
    else: # else it exists so append without writing the header
        res.head(3).to_csv(output_file, mode='a', header=False)

# SCZ analysis

In [3]:
snp_file_core_eur = pd.read_table('../dataset/preprocessed_dataset/batch2_HAI/EUR/PRS/PRSice/batch2_EUR_genotyped.50.SCZ.european.ld_EUR_EAS.snp')

In [44]:
best_SNPs=genes.SNPsFunctionalAnalysis.get_the_best_SNPs(snp_file_core_eur,threshold=.01)

In [14]:
best_SNPs

,CHR,SNP,BP,P,Base
0,1,1:2379705,2379705,1.035000e-10,1
1,1,1:30430366,30430366,1.452000e-09,1
2,1,1:44100084,44100084,2.678000e-12,1
3,1,1:73305782,73305782,4.675000e-12,1
4,1,1:73559787,73559787,3.304000e-10,1
...,...,...,...,...,...
4839,22,22:50719976,50719976,8.884000e-04,1
4840,22,22:50762615,50762615,6.841000e-04,1
4841,22,22:50975753,50975753,4.283000e-04,1
4842,22,22:51078251,51078251,6.783000e-04,1


In [28]:
best_SNPs_annot = genes.SNPsFunctionalAnalysis(snps_list=best_SNPs.SNP.to_list(),                                               bed_file='../dataset/preprocessed_dataset/batch2_HAI/EUR/batch2_EUR_genotyped.50.bed')

In [23]:
df_euro = pd.read_csv('scz/eur_cohort_segmented_scz_prs_all.csv')

In [24]:
df_euro['FID'] = df_euro['IID'] = df_euro['ID']
df_euro['sex_dummy'] = [1 if i == 'female' else 0 for i in df_euro['sex']]
df_euro[['FID','IID','Imperial 79']].to_csv('scz/phenotype_EUR.txt',index=False,sep='\t')
df_euro[['FID','IID']+[f'euro_Anc_PC{i}' for i in range(1,4)]+['TBV','GA','PMA','sex_dummy']].to_csv('scz/covariate_EUR.txt',index=False,sep='\t')

In [30]:
updated_bed_file,best_SNPs_annot.snp_association = best_SNPs_annot.do_mass_univariate_test(
    orig_bed_file=best_SNPs_annot.orig_bed_file,
    snps_list=best_SNPs_annot.snps_list,
    pheno_file=
    'scz/phenotype_EUR.txt',
    covar_file=
    'scz/covariate_EUR.txt',
    phenotype="Imperial 79")

100%|██████████████████████████████████████| 4844/4844 [00:34<00:00, 140.03it/s]


In [33]:
best_SNPs_after_thresholded = best_SNPs_annot.snp_association[best_SNPs_annot.snp_association['P']<0.05]

In [34]:
genes_ID, snp_ID = best_SNPs_annot.SNPs_annotation_to_gene(
    snps_list=best_SNPs_after_thresholded.SNP.tolist(),
    gene_build_path=
    f'../dataset/genetic_dataset/gene_build/NCBI37.3.gene.loc',
    window_size=0)

In [35]:
msigdb_dataset ='../dataset/genetic_dataset/pathway_database/MSigDB/MSigDB_custom_entrez.gmt'
background_gene = '../dataset/genetic_dataset/gene_build/NCBI37_gene_loc.txt'
SCZ_res = genes.GeneSetEnrichment.ora(msigdb_dataset,background_gene,genes_ID.Gene_ID.tolist(),disable_tqdm=False)

100%|████████████████████████████████████| 13159/13159 [01:58<00:00, 110.78it/s]
